<a href="https://colab.research.google.com/github/Frankl1sales/cnc_d/blob/main/Relat%C3%B3rio_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Algoritmo Polinômio de Lagrange
# Objetivo: Interpolar valor usando polinômio de Lagrange

# Função para calcular o polinômio de Lagrange
def lagrange_interpolation(x, y, z):
    m = len(x)  # número de pontos
    Pz = 0  # valor interpolado

    for i in range(m):
        c = 1
        d = 1
        for j in range(m):
            if i != j:
                c *= (z - x[j])
                d *= (x[i] - x[j])
        Pz += y[i] * (c / d)

    return Pz

In [ ]:
# Exemplo de uso
x = [-1, 0, 2]  # vetor de abscissas
y = [4, 1, -1]  # vetor de ordenadas
z = 0.8  # valor a interpolar

# Calculando o valor interpolado
Pz = lagrange_interpolation(x, y, z)
print(f'O valor interpolado em z={z} é Pz={Pz}')

O valor interpolado em z=0.8 é Pz=-0.43999999999999995


In [ ]:
def newton_interpolation(x, y, z):
    m = len(x)  # número de pontos
    Dely = y.copy()  # inicializa o vetor das diferenças divididas com os valores de y

    # Construção das diferenças divididas
    for i in range(1, m):
        for k in range(m - 1, i - 1, -1):
            Dely[k] = (Dely[k] - Dely[k - 1]) / (x[k] - x[k - i])

    # Avaliação do polinômio pelo método de Horner
    Pz = Dely[m - 1]
    for i in range(m - 2, -1, -1):
        Pz = Pz * (z - x[i]) + Dely[i]

    return Pz

In [ ]:
 # Exemplo de uso
x = [1, 2, 3]  # vetor de abscissas
y = [2, 3, 5]  # vetor de ordenadas
z = 2.5  # valor a interpolar

# Calculando o valor interpolado
Pz = newton_interpolation(x, y, z)
print(f'O valor interpolado em z={z} é Pz={Pz}')

O valor interpolado em z=2.5 é Pz=3.875
